In [1]:
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install prettytable


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 57.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp312-cp312-linux_x86_64.whl size=1196046 sha256=c5a2dc333b7ea51148c099f159b2fbe62e7cac77838fc62aac136bdf9e3dd841
  Stored in directory: /root/.cache/pip/wheels/b3/1c/42/0e26b8d512adc6bce10ff71a05229366b4ccec641cd3b42111
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.44
    Uninstalling SQLAlchemy-2.0.44:
      Successfully uninstalled SQLAlchemy-2.0.44
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.19.0 requires sqlalchemy<3.0.0,>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
alembic 1.17.2 requires 

In [2]:
%load_ext sql


In [4]:
%sql sqlite:///my_spacex.db


In [5]:
import pandas as pd

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv"

df = pd.read_csv(url)
df.head()


,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [9]:
import sqlalchemy

# Create a SQLAlchemy engine for the SQLite database
engine = sqlalchemy.create_engine('sqlite:///my_spacex.db')

# Write the dataframe to the SQL table using the engine
df.to_sql("SPACEXTABLE", con=engine, if_exists='replace', index=False)

101

In [11]:
%sql DROP TABLE IF EXISTS SPACEXTABLE_CLEANED;


 * sqlite:///my_spacex.db
Done.


[]

In [15]:
%sql CREATE TABLE SPACEXTABLE_CLEANED AS SELECT * FROM SPACEXTABLE WHERE Date IS NOT NULL;


 * sqlite:///my_spacex.db
(sqlite3.OperationalError) table SPACEXTABLE_CLEANED already exists
[SQL: CREATE TABLE SPACEXTABLE_CLEANED AS SELECT * FROM SPACEXTABLE WHERE Date IS NOT NULL;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [1]:
!pip install "prettytable<1.0"


In [2]:
%load_ext sql
%sql sqlite:///my_spacex.db


In [3]:
%%sql
SELECT DISTINCT Launch_Site
FROM SPACEXTABLE;


 * sqlite:///my_spacex.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [4]:
%%sql
SELECT * FROM SPACEXTABLE_CLEANED
WHERE Launch_Site LIKE "CCA%"
LIMIT 5;


 * sqlite:///my_spacex.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [5]:
%%sql
SELECT SUM(Payload_Mass__kg_) AS Total_Payload
FROM SPACEXTABLE_CLEANED
WHERE Customer = "NASA (CRS)";


 * sqlite:///my_spacex.db
Done.


Total_Payload
45596


In [6]:
%%sql
SELECT AVG(Payload_Mass__kg_) AS Average_Payload
FROM SPACEXTABLE_CLEANED
WHERE Booster_Version = "F9 v1.1";



 * sqlite:///my_spacex.db
Done.


Average_Payload
2928.4


In [7]:
%%sql
SELECT MIN(Date) AS First_Success_Ground
FROM SPACEXTABLE_CLEANED
WHERE Landing_Outcome = "Success (ground pad)";


 * sqlite:///my_spacex.db
Done.


First_Success_Ground
2015-12-22


In [8]:
%%sql
SELECT DISTINCT Booster_Version
FROM SPACEXTABLE_CLEANED
WHERE Landing_Outcome = "Success (drone ship)"
  AND Payload_Mass__kg_ > 4000
  AND Payload_Mass__kg_ < 6000;


 * sqlite:///my_spacex.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [9]:
%%sql
SELECT Landing_Outcome, COUNT(*) AS Count
FROM SPACEXTABLE_CLEANED
GROUP BY Landing_Outcome;


 * sqlite:///my_spacex.db
Done.


Landing_Outcome,Count
Controlled (ocean),5
Failure,3
Failure (drone ship),5
Failure (parachute),2
No attempt,21
No attempt,1
Precluded (drone ship),1
Success,38
Success (drone ship),14
Success (ground pad),9


In [10]:
%%sql
SELECT Booster_Version, Payload_Mass__kg_
FROM SPACEXTABLE_CLEANED
WHERE Payload_Mass__kg_ = (
    SELECT MAX(Payload_Mass__kg_) FROM SPACEXTABLE_CLEANED
);


 * sqlite:///my_spacex.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


In [11]:
%%sql
SELECT
    substr(Date, 6, 2) AS Month,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM SPACEXTABLE_CLEANED
WHERE substr(Date, 1, 4) = "2015";


 * sqlite:///my_spacex.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
02,Controlled (ocean),F9 v1.1 B1013,CCAFS LC-40
03,No attempt,F9 v1.1 B1014,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40
04,No attempt,F9 v1.1 B1016,CCAFS LC-40
06,Precluded (drone ship),F9 v1.1 B1018,CCAFS LC-40
12,Success (ground pad),F9 FT B1019,CCAFS LC-40


In [12]:
%%sql
SELECT Landing_Outcome, COUNT(*) AS Count
FROM SPACEXTABLE_CLEANED
WHERE Date BETWEEN "2010-06-04" AND "2017-03-20"
GROUP BY Landing_Outcome
ORDER BY Count DESC;


 * sqlite:///my_spacex.db
Done.


Landing_Outcome,Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
